# Web Scraping for Different Types of Roller Coasters

In [98]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd, numpy as np, datetime as dt
from pprint import pprint
import json
import re
import numpy as np


In [99]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_mac64.zip
Driver has been saved in cache [/Users/bt4ever/.wdm/drivers/chromedriver/mac64/91.0.4472.101]


In [100]:
coaster_types = {}

In [101]:
url='https://rcdb.com/r.htm?ot=2'
browser.visit(url)

In [102]:
html = browser.html
soup = bs(html, 'html.parser')
#coaster_info = soup.select_one('div', class_='list_text')
#coster_info
table = pd.read_html(url)
table


[            0                      1
 0   Criteria:        Roller Coasters
 1      Found:  10671 (Page 1 of 445)
 2  Sorted By:         Roller Coaster,
     Unnamed: 0                Roller Coaster                   Amusement Park  \
 0          NaN                â¬uro-Coaster                    Wiener Prater   
 1          NaN                â¬uro Coaster               Funland Theme Park   
 2          NaN                          1066                        Festyland   
 3          NaN   10 Inversion Roller Coaster               Chimelong Paradise   
 4          NaN  1970 Galaxy Rip Tide Coaster    Swampy Jack's Wongo Adventure   
 5          NaN                 2 Way Coaster                  Tokyo Dome City   
 6          NaN               3 Loops Coaster                     Central Park   
 7          NaN         3 Ring Roller Coaster                      Daguan Park   
 8          NaN         3 Ring Roller Coaster   Middle Zhejiang Amusement City   
 9          NaN            

In [103]:
url = f'https://rcdb.com/r.htm?ot=2'
coasters = soup.find_all("table")
print("Number of tables on site: ",len(coasters))

Number of tables on site:  2


In [104]:
rc_complete = []

for i in range(1,446):
    url = f'https://rcdb.com/r.htm?ot=2&page={i}'
    table1 = coasters[1]
    coasters = soup.find_all("table")
    #print(len(coasters)) #Output 3

# Lets go ahead and scrape first table with HTML code gdp[0]
    table1 = gdp[1]

# the head will form our column names
    body = table1.find_all("tr")

# Head values (Column names) are the first items of the body list
    head = body[0] # 0th item is the header row
    body_rows = body[1:] # All other items becomes the rest of the rows

# Lets now iterate through the head HTML code and make list of clean headings

# Declare empty list to keep Columns names
    headings = []
    for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
        item = (item.text).rstrip("\n")
    # append the clean column name to headings
        headings.append(item)
    #print(headings)

# Next is now to loop though the rest of the rows

#print(body_rows[0])
    all_rows = [] # will be a list for list for all rows
    for row_num in range(len(body_rows)): # A row at a time
        row = [] # this will old entries for one row
        for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
            row.append(aa)
    # append one row to all_rows
        all_rows.append(row)

# We can now use the data on all_rowsa and headings to make a table
# all_rows becomes our data and headings the column names
    complete_df = pd.DataFrame(data=all_rows,columns=headings)
    rc_complete.append(complete_df)
    rc_complete_df = pd.concat(rc_complete, ignore_index=True)
    rc_final = rc_complete_df.drop(columns=[''])
rc_final

,Roller Coaster,Amusement Park,Type,Design,Status,Opened
0,€uro-Coaster,Wiener Prater,Steel,Suspended,-,5/29/2020
1,€uro Coaster,Funland Theme Park,Steel,Sit Down,Operating,2021
2,1066,Festyland,Steel,Sit Down,Operating,3/27/2005
3,10 Inversion Roller Coaster,Chimelong Paradise,Steel,Sit Down,Operating,2/2006
4,1970 Galaxy Rip Tide Coaster,Swampy Jack's Wongo Adventure,Steel,Sit Down,Relocated,6/27/2014
...,...,...,...,...,...,...
10675,7 Picos,Parque de Atracciones de Madrid,Steel,Sit Down,-,5/15/1969
10676,88 Train,Children's Grand Park,Steel,Sit Down,-,1984
10677,88 Train,Amusement Land,Steel,Sit Down,-,-
10678,Abandon Mine,Uncle Bernie's Theme Park,Steel,Sit Down,Operating,2002


In [105]:
rc_final['Opened'] = rc_final['Opened'].str[-4:]

In [106]:
rc_done = rc_final.replace('-',np.nan)
rc_done

,Roller Coaster,Amusement Park,Type,Design,Status,Opened
0,€uro-Coaster,Wiener Prater,Steel,Suspended,NaN,2020
1,€uro Coaster,Funland Theme Park,Steel,Sit Down,Operating,2021
2,1066,Festyland,Steel,Sit Down,Operating,2005
3,10 Inversion Roller Coaster,Chimelong Paradise,Steel,Sit Down,Operating,2006
4,1970 Galaxy Rip Tide Coaster,Swampy Jack's Wongo Adventure,Steel,Sit Down,Relocated,2014
...,...,...,...,...,...,...
10675,7 Picos,Parque de Atracciones de Madrid,Steel,Sit Down,NaN,1969
10676,88 Train,Children's Grand Park,Steel,Sit Down,NaN,1984
10677,88 Train,Amusement Land,Steel,Sit Down,NaN,NaN
10678,Abandon Mine,Uncle Bernie's Theme Park,Steel,Sit Down,Operating,2002


In [107]:
rc_done.to_json("rcdb.json", orient='columns')


In [108]:
rc_done.to_json("rcdb_split.json", orient='split')

In [109]:
rc_done.to_csv("rcdb_with_years.csv", index=False)

In [110]:
rc_done

,Roller Coaster,Amusement Park,Type,Design,Status,Opened
0,€uro-Coaster,Wiener Prater,Steel,Suspended,NaN,2020
1,€uro Coaster,Funland Theme Park,Steel,Sit Down,Operating,2021
2,1066,Festyland,Steel,Sit Down,Operating,2005
3,10 Inversion Roller Coaster,Chimelong Paradise,Steel,Sit Down,Operating,2006
4,1970 Galaxy Rip Tide Coaster,Swampy Jack's Wongo Adventure,Steel,Sit Down,Relocated,2014
...,...,...,...,...,...,...
10675,7 Picos,Parque de Atracciones de Madrid,Steel,Sit Down,NaN,1969
10676,88 Train,Children's Grand Park,Steel,Sit Down,NaN,1984
10677,88 Train,Amusement Land,Steel,Sit Down,NaN,NaN
10678,Abandon Mine,Uncle Bernie's Theme Park,Steel,Sit Down,Operating,2002


In [111]:
rc_done['Opened'] = pd.to_datetime(rc_done['Opened'], format='%Y', errors='coerce')
rc_done.to_csv("rcdb_w_years_dtformat.csv", index=False)

In [112]:
rc_done.to_json("rcdb_dtformat.json")


In [113]:
rc_done.to_json("rcdb_dtformat_split.json", orient='split')

In [114]:
#Speed Record Holders
url = 'https://rcdb.com/rhr.htm'

dfs = pd.read_html(url)
speed_df = dfs[1]
speed_df

,Rank,Speed,Roller Coaster,Amusement Park (Location),Opened
0,1st,149.1 mph,Formula Rossa,"Ferrari World Abu Dhabi (Abu Dhabi, Abu Dhabi,...",NaN
1,2nd,128 mph,Kingda Ka,"Six Flags Great Adventure (Jackson, New Jersey...",NaN
2,3rd,120 mph,Top Thrill Dragster,"Cedar Point (Sandusky, Ohio, United States)",NaN
3,4th,111.8 mph,Do-DodonpaRed Force,"Fuji-Q Highland (Fujiyoshida, Yamanashi, Japan...",NaN
4,5th,100 mph,Superman: Escape from Krypton,"Six Flags Magic Mountain (Valencia, California...",NaN
5,*,99.4 mph,Ring Racer,"NÃ¼rburgring (NÃ¼rburg, Rhineland-Palatinate, ...",NaN
6,6th,95 mph,Steel Dragon 2000Fury 325,"Nagashima Spa Land (Kuwana, Mie, Japan)Carowin...",NaN
7,7th,93 mph,Millennium Force,"Cedar Point (Sandusky, Ohio, United States)",NaN
8,8th,92 mph,Leviathan,"Canada's Wonderland (Vaughan, Ontario, Canada)",NaN
9,9th,91 mph,Orion,"Kings Island (Mason, Ohio, United States)",NaN


In [115]:

#Height Record Holders
url = 'https://rcdb.com/rhr.htm?m=1'

dfs = pd.read_html(url)
height_df = dfs[1]
height_df

,Rank,Height,Roller Coaster,Amusement Park (Location),Opened
0,1st,456 ft,Kingda Ka,"Six Flags Great Adventure (Jackson, New Jersey...",NaN
1,2nd,420 ft,Top Thrill Dragster,"Cedar Point (Sandusky, Ohio, United States)",NaN
2,3rd,415 ft,Superman: Escape from Krypton,"Six Flags Magic Mountain (Valencia, California...",NaN
3,4th,367.3 ft,Red Force,"Ferrari Land (Salou, Tarragona, Catalonia, Spain)",NaN
4,5th,325 ft,Fury 325,"Carowinds (Charlotte, North Carolina, United S...",NaN
5,6th,318.3 ft,Steel Dragon 2000,"Nagashima Spa Land (Kuwana, Mie, Japan)",NaN
6,7th,310 ft,Millennium Force,"Cedar Point (Sandusky, Ohio, United States)",NaN
7,8th,306 ft,Leviathan,"Canada's Wonderland (Vaughan, Ontario, Canada)",NaN
8,9th,305 ft,Intimidator 305,"Kings Dominion (Doswell, Virginia, United States)",NaN
9,10th,287 ft,Orion,"Kings Island (Mason, Ohio, United States)",NaN


In [116]:
#Length 
url = 'https://rcdb.com/rhr.htm?m=2'

dfs = pd.read_html(url)
length_df = dfs[1]
length_df

,Rank,Length,Roller Coaster,Amusement Park (Location),Opened
0,1st,8133.2 ft,Steel Dragon 2000,"Nagashima Spa Land (Kuwana, Mie, Japan)",NaN
1,*,7442 ft,Ultimate,"Lightwater Valley (Ripon, North Yorkshire, Eng...",NaN
2,2nd,7359 ft,Beast,"Kings Island (Mason, Ohio, United States)",NaN
3,3rd,6708.7 ft,Fujiyama,"Fuji-Q Highland (Fujiyoshida, Yamanashi, Japan)",NaN
4,4th,6602 ft,Fury 325,"Carowinds (Charlotte, North Carolina, United S...",NaN
5,5th,6595 ft,Millennium Force,"Cedar Point (Sandusky, Ohio, United States)",NaN
6,6th,6561.7 ft,Formula Rossa,"Ferrari World Abu Dhabi (Abu Dhabi, Abu Dhabi,...",NaN
7,7th,6442 ft,Voyage,"Holiday World (Santa Claus, Indiana, United St...",NaN
8,8th,6072 ft,Incredicoaster,"Disney California Adventure Park (Anaheim, Cal...",NaN
9,9th,5843 ft,Desperado,"Buffalo Bill's Resort & Casino (Primm, Nevada,...",NaN


In [117]:
length_df[['Amusement Park','Location']] = length_df['Amusement Park (Location)'].str.split('(',expand=True)
length_df

,Rank,Length,Roller Coaster,Amusement Park (Location),Opened,Amusement Park,Location
0,1st,8133.2 ft,Steel Dragon 2000,"Nagashima Spa Land (Kuwana, Mie, Japan)",NaN,Nagashima Spa Land,"Kuwana, Mie, Japan)"
1,*,7442 ft,Ultimate,"Lightwater Valley (Ripon, North Yorkshire, Eng...",NaN,Lightwater Valley,"Ripon, North Yorkshire, England, United Kingdom)"
2,2nd,7359 ft,Beast,"Kings Island (Mason, Ohio, United States)",NaN,Kings Island,"Mason, Ohio, United States)"
3,3rd,6708.7 ft,Fujiyama,"Fuji-Q Highland (Fujiyoshida, Yamanashi, Japan)",NaN,Fuji-Q Highland,"Fujiyoshida, Yamanashi, Japan)"
4,4th,6602 ft,Fury 325,"Carowinds (Charlotte, North Carolina, United S...",NaN,Carowinds,"Charlotte, North Carolina, United States)"
5,5th,6595 ft,Millennium Force,"Cedar Point (Sandusky, Ohio, United States)",NaN,Cedar Point,"Sandusky, Ohio, United States)"
6,6th,6561.7 ft,Formula Rossa,"Ferrari World Abu Dhabi (Abu Dhabi, Abu Dhabi,...",NaN,Ferrari World Abu Dhabi,"Abu Dhabi, Abu Dhabi, United Arab Emirates)"
7,7th,6442 ft,Voyage,"Holiday World (Santa Claus, Indiana, United St...",NaN,Holiday World,"Santa Claus, Indiana, United States)"
8,8th,6072 ft,Incredicoaster,"Disney California Adventure Park (Anaheim, Cal...",NaN,Disney California Adventure Park,"Anaheim, California, United States)"
9,9th,5843 ft,Desperado,"Buffalo Bill's Resort & Casino (Primm, Nevada,...",NaN,Buffalo Bill's Resort & Casino,"Primm, Nevada, United States)"
